## Similarity search for text to text

##### First trial for text similarity search for word-->vectors and vestor search with corpus of description words
##### here we trying with haystack. Links given for exploration

## Haystack v0.1
#### https://haystack.deepset.ai/components/document-store

In [5]:
import pandas as pd
df = pd.read_csv('City_data1.csv')

In [9]:
# !pip install git+https://github.com/deepset-ai/haystack.git
# !pip install sentence-transformers

In [3]:
import os

import pandas as pd
import numpy as np

from haystack.document_store.faiss import FAISSDocumentStore
from haystack.retriever.dense import EmbeddingRetriever

/Users/apple/opt/anaconda3/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [6]:
df['text'] = df['desc'] + df['sights']#+ df['Best_time']
df2 = df[['desc','Best_time','city']]

In [7]:
# API fit
document_store_faiss = FAISSDocumentStore(faiss_index_factory_str="Flat",return_embedding=True)

retriever_faiss = EmbeddingRetriever(document_store_faiss, embedding_model='distilroberta-base-msmarco-v2',model_format='sentence_transformers')

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.retriever.dense -  Init retriever using embeddings of model distilroberta-base-msmarco-v2


WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [10]:
#document setup for haystack
from typing import List
from haystack import Document
document_store_faiss.delete_documents()
cities = list(df["city"].values)
descs = list(df["desc"].values)
documents: List[Document] = []
for city, desc in zip(cities, descs):
    documents.append(
        Document(
            content=desc,
            meta={
                "name": city 
            }
        )
    )

In [11]:
#document write
document_store_faiss.write_documents(documents)

In [12]:
#document update
document_store_faiss.update_embeddings(retriever=retriever_faiss)

INFO - haystack.document_store.faiss -  Updating embeddings for 100 docs...


In [13]:
#custom function for predictions similarity
def get_results(query, retriever, n_docs = 6):
    
      return [item for item in retriever.retrieve(q, top_k = n_docs)]

In [14]:
# time for some query
q = 'somewhere romantic '
print('-'*100)
print('Results: ')
print('-'*75)
res = get_results(q, retriever_faiss) 
for r in res:
    print(r)

----------------------------------------------------------------------------------------------------
Results: 
---------------------------------------------------------------------------



<Document: id=e8b27998e0621c0938e8faeb0e9eb815, content='The ultimate noir experience for art lovers.Germany's capital, with the Reichstag, Brandenburg Gate, ...'>
<Document: id=e9bb3fad086553a5f011c00950ff2677, content='The eternal city is a heartland for food and history lovers.The sprawling city of Rome remains one o ...'>
<Document: id=acca9de40f888dc0f5fef2ae0f2c1211, content='Melbourne, Live music, renowned coffee and top-notch shopping in one city. The vibrant coastal capit ...'>
<Document: id=e05db8fe73a1eb7d5f6367197dad541e, content='Manila, Ideal for family holidays, Manila is a city for kids of all ages. The Philippines’ bustling, ...'>
<Document: id=21cc0561391231c4994dcc7f5de6f31b, content='There’s more to be experienced than coffee and grunge in this ever-evolving city.In Seattle you simp ...'>
<Document: id=2bf36213528ebc1e73019dad91138d9b, content='Male, The Maldive capital is your gateway to world-famous beaches. The island capital of the Maldive ...'>


In [15]:
## trials 
# not the best similarity search